In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 2.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'detectron2'...
remote: Enumerating objects: 15549, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (211/211), done.
remote: Total 15549 (delta 92), reused 202 (delta 63), pack-reused 15275
Receiving objects: 100% (15549/15549), 6.41 MiB | 12.37 MiB/s, done.
Resolving deltas: 100% (11203/11203), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.9 MB/s eta 0:00:

In [ ]:
import logging
import os
from detectron2.utils.logger import setup_logger
setup_logger()
import cv2
from collections import OrderedDict
from detectron2 import model_zoo
from detectron2.data.catalog import DatasetCatalog
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, hooks, launch
from detectron2.evaluation import CityscapesInstanceEvaluator
from detectron2.evaluation import CityscapesSemSegEvaluator
from detectron2.evaluation import COCOEvaluator
from detectron2.evaluation import COCOPanopticEvaluator
from detectron2.evaluation import DatasetEvaluators
from detectron2.evaluation import LVISEvaluator
from detectron2.evaluation import PascalVOCDetectionEvaluator
from detectron2.evaluation import SemSegEvaluator
from detectron2.evaluation import verify_results
from detectron2.modeling import GeneralizedRCNNWithTTA
from detectron2.data.datasets import register_coco_instances
import random
from detectron2.utils.visualizer import Visualizer
from detectron2.engine import DefaultPredictor
import cv2
from demo.predictor import VisualizationDemo
from detectron2.utils.visualizer import ColorMode

In [ ]:
register_coco_instances("my_dataset_train", {}, "/content/drive/MyDrive/MLE_Group14/dsb2018_t.v1i.coco-segmentation/train/_annotations.coco.json", "/content/drive/MyDrive/MLE_Group14/dsb2018_t.v1i.coco-segmentation/train")
register_coco_instances("my_dataset_val", {}, "/content/drive/MyDrive/MLE_Group14/dsb2018_t.v1i.coco-segmentation/valid/_annotations.coco.json", "/content/drive/MyDrive/MLE_Group14/dsb2018_t.v1i.coco-segmentation/valid")
register_coco_instances("my_dataset_test", {},"/content/drive/MyDrive/MLE_Group14/dsb2018_t.v1i.coco-segmentation/test/_annotations.coco.json", "/content/drive/MyDrive/MLE_Group14/dsb2018_t.v1i.coco-segmentation/test")
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train").set(
    thing_classes=['nuclei-Jf4c', 'Nuclei',])
dataset_dicts = DatasetCatalog.get("my_dataset_train")

[04/24 15:53:17 d2.data.datasets.coco]: Loading /content/drive/MyDrive/MLE_Group14/dsb2018_t.v1i.coco-segmentation/train/_annotations.coco.json takes 8.29 seconds.
WARNING [04/24 15:53:17 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/24 15:53:17 d2.data.datasets.coco]: Loaded 498 images in COCO format from /content/drive/MyDrive/MLE_Group14/dsb2018_t.v1i.coco-segmentation/train/_annotations.coco.json


In [ ]:
def build_evaluator(cfg, dataset_name, output_folder=None):
    if output_folder is None:
        output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
    evaluator_list = []
    evaluator_type = MetadataCatalog.get(dataset_name).evaluator_type
    if evaluator_type in ["sem_seg", "coco_panoptic_seg"]:
        evaluator_list.append(
            SemSegEvaluator(
                dataset_name,
                distributed=True,
                output_dir=output_folder,
            )
        )
    if evaluator_type in ["coco", "coco_panoptic_seg"]:
        evaluator_list.append(COCOEvaluator(dataset_name, output_dir=output_folder))
    if evaluator_type == "coco_panoptic_seg":
        evaluator_list.append(COCOPanopticEvaluator(dataset_name, output_folder))
    if evaluator_type == "cityscapes_instance":
        return CityscapesInstanceEvaluator(dataset_name)
    if evaluator_type == "cityscapes_sem_seg":
        return CityscapesSemSegEvaluator(dataset_name)
    elif evaluator_type == "pascal_voc":
        return PascalVOCDetectionEvaluator(dataset_name)
    elif evaluator_type == "lvis":
        return LVISEvaluator(dataset_name, output_dir=output_folder)
    if len(evaluator_list) == 0:
        raise NotImplementedError(
            "no Evaluator for the dataset {} with the type {}".format(dataset_name, evaluator_type)
        )
    elif len(evaluator_list) == 1:
        return evaluator_list[0]
    return DatasetEvaluators(evaluator_list)

In [ ]:
class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return build_evaluator(cfg, dataset_name, output_folder)

    @classmethod
    def test_with_TTA(cls, cfg, model):
        logger = logging.getLogger("detectron2.trainer")
        logger.info("Running inference with test-time augmentation ...")
        model = GeneralizedRCNNWithTTA(cfg, model)
        evaluators = [
            cls.build_evaluator(
                cfg, name, output_folder=os.path.join(cfg.OUTPUT_DIR, "inference_TTA")
            )
            for name in cfg.DATASETS.TEST
        ]
        res = cls.test(cfg, model, evaluators)
        res = OrderedDict({k + "_TTA": v for k, v in res.items()})
        return res

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 1
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

cfg.INPUT.CROP.ENABLED = True
cfg.INPUT.MAX_SIZE_TRAIN = 768
cfg.INPUT.MAX_SIZE_TEST = 768
cfg.INPUT.MIN_SIZE_TRAIN = 512
cfg.INPUT.MIN_SIZE_TEST = 512
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg.MODEL.RETINANET.NUM_CLASSES = 3
ITERS_IN_ONE_EPOCH = int(9000 / cfg.SOLVER.IMS_PER_BATCH)

# cfg.SOLVER.MAX_ITER = (ITERS_IN_ONE_EPOCH * 12) - 1
cfg.SOLVER.MAX_ITER = ITERS_IN_ONE_EPOCH- 1
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.MOMENTUM = 0.9
cfg.SOLVER.WEIGHT_DECAY = 0.0001
cfg.SOLVER.WEIGHT_DECAY_NORM = 0.0
cfg.SOLVER.GAMMA = 0.1
cfg.SOLVER.STEPS = (7000,)
cfg.SOLVER.WARMUP_FACTOR = 1.0 / 1000
cfg.SOLVER.WARMUP_ITERS = 1000

cfg.SOLVER.WARMUP_METHOD = "linear"

cfg.TEST.EVAL_PERIOD = 500

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = Trainer(cfg)
trainer.resume_or_load(resume=False)
if cfg.TEST.AUG.ENABLED:
    trainer.register_hooks(
        [hooks.EvalHook(0, lambda: trainer.test_with_TTA(cfg, trainer.model))]
    )

[04/24 16:38:56 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


In [ ]:
trainer.train()

[04/24 16:39:01 d2.engine.train_loop]: Starting training from iteration 0


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[04/24 16:39:50 d2.utils.events]:  eta: 0:22:20  iter: 19  total_loss: 4.895  loss_cls: 1.04  loss_box_reg: 0.744  loss_mask: 0.686  loss_rpn_cls: 2.077  loss_rpn_loc: 0.3817    time: 2.4427  last_time: 2.3539  data_time: 0.8736  last_data_time: 0.6086   lr: 8.7086e-05  max_mem: 7470M
[04/24 16:40:39 d2.utils.events]:  eta: 0:21:17  iter: 39  total_loss: 2.465  loss_cls: 0.5945  loss_box_reg: 0.7515  loss_mask: 0.54  loss_rpn_cls: 0.284  loss_rpn_loc: 0.3395    time: 2.4443  last_time: 3.5608  data_time: 0.8765  last_data_time: 1.9482   lr: 0.00017612  max_mem: 7470M
[04/24 16:41:25 d2.utils.events]:  eta: 0:19:41  iter: 59  total_loss: 1.976  loss_cls: 0.4039  loss_box_reg: 0.7179  loss_mask: 0.4049  loss_rpn_cls: 0.1677  loss_rpn_loc: 0.2678    time: 2.3888  last_time: 2.6875  data_time: 0.7100  last_data_time: 1.1782   lr: 0.00026516  max_mem: 7470M
[04/24 16:42:13 d2.utils.events]:  eta: 0:18:47  iter: 79  total_loss: 1.687  loss_cls: 0.3057  loss_box_reg: 0.6237  loss_mask: 0.3387

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[04/24 16:59:26 d2.evaluation.evaluator]: Inference done 11/166. Dataloading: 0.2302 s/iter. Inference: 0.0842 s/iter. Eval: 0.0149 s/iter. Total: 0.3293 s/iter. ETA=0:00:51
[04/24 16:59:32 d2.evaluation.evaluator]: Inference done 24/166. Dataloading: 0.2854 s/iter. Inference: 0.0838 s/iter. Eval: 0.0203 s/iter. Total: 0.3897 s/iter. ETA=0:00:55
[04/24 16:59:37 d2.evaluation.evaluator]: Inference done 36/166. Dataloading: 0.3066 s/iter. Inference: 0.0846 s/iter. Eval: 0.0192 s/iter. Total: 0.4105 s/iter. ETA=0:00:53
[04/24 16:59:46 d2.evaluation.evaluator]: Inference done 42/166. Dataloading: 0.4965 s/iter. Inference: 0.0825 s/iter. Eval: 0.0172 s/iter. Total: 0.5965 s/iter. ETA=0:01:13
[04/24 16:59:52 d2.evaluation.evaluator]: Inference done 55/166. Dataloading: 0.4452 s/iter. Inference: 0.0826 s/iter. Eval: 0.0199 s/iter. Total: 0.5478 s/iter. ETA=0:01:00
[04/24 16:59:57 d2.evaluation.evaluator]: Inference done 60/166. Dataloading: 0.4943 s/iter. Inference: 0.0823 s/iter. Eval: 0.018

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[04/24 17:03:39 d2.evaluation.evaluator]: Inference done 11/166. Dataloading: 0.0011 s/iter. Inference: 0.0567 s/iter. Eval: 0.0112 s/iter. Total: 0.0690 s/iter. ETA=0:00:10
[04/24 17:03:44 d2.evaluation.evaluator]: Inference done 68/166. Dataloading: 0.0015 s/iter. Inference: 0.0596 s/iter. Eval: 0.0252 s/iter. Total: 0.0864 s/iter. ETA=0:00:08
[04/24 17:03:49 d2.evaluation.evaluator]: Inference done 116/166. Dataloading: 0.0026 s/iter. Inference: 0.0624 s/iter. Eval: 0.0291 s/iter. Total: 0.0942 s/iter. ETA=0:00:04
[04/24 17:03:54 d2.evaluation.evaluator]: Total inference time: 0:00:15.246301 (0.094698 s / iter per device, on 1 devices)
[04/24 17:03:54 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:09 (0.061813 s / iter per device, on 1 devices)
[04/24 17:03:54 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[04/24 17:03:54 d2.evaluation.coco_evaluation]: Saving results to ./output/inference/coco_instances_results.json
[04/24 17:03:54 d2.evalu